<a href="https://colab.research.google.com/github/fighting58/Colab/blob/main/districtcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

requests 이용하여 법정동코드 전체자료 다운로드

In [ ]:
import requests
import io
import zipfile
import os

class CodeGoKr:
    def __init__(self):
        self.__url ='https://www.code.go.kr/etc/codeFullDown.do?codeseId=00002'

        self.__header = {'Host': 'www.code.go.kr',
                        'Connection': 'keep-alive',
                        'Content-Length': '14',
                        'Cache-Control': 'max-age=0',
                        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
                        'sec-ch-ua-mobile': '?0',
                        'sec-ch-ua-platform': "Windows",
                        'Upgrade-Insecure-Requests': '1',
                        'Origin': 'https://www.code.go.kr',
                        'Content-Type': 'application/x-www-form-urlencoded',
                        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
                        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                        'Sec-Fetch-Site': 'same-origin',
                        'Sec-Fetch-Mode': 'navigate',
                        'Sec-Fetch-User': '?1',
                        'Sec-Fetch-Dest': 'iframe',
                        'Referer': 'https://www.code.go.kr/stdcodesrch/codeAllDownloadL.do',
                        'Accept-Encoding': 'gzip, deflate, br',
                        'Accept-Language': 'ko,en-US;q=0.9,en;q=0.8',
                        'Cookie': 'codeCount=20220103; JSESSIONID=eP5H5+3-UnWLRN3xgOOT4eU7.CODROOTserver1; clientid=040011633063; GPKISecureWebSession=gsrmGc1RCWu3snviavdB'
                        }

        self.__payload = {'codeseld': '00002'}
        self.__work_path='/content/drive/MyDrive/Colab Notebooks'
        self.filename = self.__work_path + '/법정동코드 전체자료.zip' 
        self.__tries = 0

    def __getZip(self):
        """ 행정표준코드관리시스템에 접속하여 법정동코드를 다운로드 """
        flag = 0
        try:
            response = requests.post(self.__url, headers=self.__header, data=self.__payload, stream=True)
            response.encoding = response.apparent_encoding
            self.__tries += 1
            flag = 1
        except requests.exceptions.ConnectionError as e:
            print(e)
        finally:
            if flag:      
                with open(self.filename,"wb") as zip:
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            zip.write(chunk)
            elif self.__tries > 5:
                print("Connection Fail - Try Again")
            else:
                self.__getzip()
                

    def __unzip(self, source_file, dest_path):
        """압축 해제 후 그 첫번째 파일명을 반환"""

        name = ''
        with zipfile.ZipFile(source_file, 'r') as zf:
            zipInfo = zf.infolist()
            for member in zipInfo:
                try:
                    # print(member.filename.encode('cp437').decode('euc-kr', 'ignore'))
                    member.filename = member.filename.encode('cp437').decode('euc-kr', 'ignore')
                    zf.extract(member, dest_path)
                except:
                    print(source_file)
                    raise Exception('what?!')
                    return 
            name = zipInfo[0].filename
        return name

    def getDB(self):
        """ 일괄 처리 """

        self.__getZip()
        if os.path.exists(self.filename):
            textfile = self.__unzip(self.filename, self.__work_path)
            if not textfile:
                print("DB 생성에 실패하였습니다.")
                return
            # 성공하면 zip파일 삭제
            os.remove(self.filename)
            print(f"{textfile}: 생성 성공")
            # 파일명 반환
            return os.path.join(self.__work_path, textfile)
        return

법정동 코드 다운로드: www.code.go.kr 
 <ul><li>법정동코드 전체자료.txt</li></ul>

In [ ]:
import pandas as pd
import os

class DistrictCode:
    def __init__(self, sido='41'):
         self.__sido = sido
         self.__pickle = None
         self.__path = None
         self.__df = None
         self.set_path()

    def set_path(self, path='/content/drive/MyDrive/Colab Notebooks'):
         self.__path = path
         self.__df = self.__get_df()  

    def __get_df(self):
         filename = f'District{self.__sido}.pickle'
         self.__pickle = os.path.join(self.__path, filename)
         if os.path.exists(self.__pickle):
             return pd.read_pickle(self.__pickle)
         return self.__create_df()

    def __create_df(self):
         # 법정동코드 파일 경로
         txtfile = os.path.join(self.__path, '법정동코드 전체자료.txt') 
         if not os.path.exists(txtfile):
             print('법정동코드 전체자료.txt: File not Found')
             print('Try to download: www.code.go.kr')
             code_go_kr = CodeGoKr()
             download = code_go_kr.getDB()
             if not download:
                 print("Download 실패")
                 return pd.DataFrame()
         # 데이터프레임 생성
         df = pd.read_csv(txtfile, encoding='euc-kr', sep='\t', 
                          dtype=str, names=['cd', 'cd_name', 'exists'])
         # 현존 코드만 추출
         df = df[df['exists'] == '존재']
         # 지정된 시도 코드만 추출
         df = df[df['cd'].str.startswith(self.__sido)]
         # 행정구역명에서 공백을 제거한 컬럼 추가
         df['nm'] = [x.replace(' ', '') for x in df['cd_name']]
         # 추후 빠른 변환을 위해 데이터프레임을 피클로 저장
         if not df.empty:
             df.to_pickle(self.__pickle)
         return df
     
    def search(self, emd):
         # 읍면동 이름에서 공백제거
         emd = emd.replace(' ', '')
         result = self.__df[self.__df['nm'].str.contains(emd)]['cd']
         # 검색내용이 없을 때
         if result.empty:
             return 'NotFound'
         # 검색 결과 중 첫번째 결과값 반환
         # 검색 정확도를 높이려면 상세한 행정구역 입력
         return result.values[0]

    def emd(self, pnu):
         # 10자 이내이거나 숫자가 아닌 경우
         if len(pnu)<10 or not pnu.isnumeric():
             return 'CD_error'
         # 입력값의 10자리만 추출
         cd = pnu[:10]     
         result = self.__df[self.__df['cd'] == cd]['cd_name']
         if result.empty: 
             return 'NotFound'
         return result.values[0]

    def make_pnu(self, emd, jibun):
         # 행정구역 검색 실패시 오류 메시지 반환
         cd = self.search(emd)
         if cd == 'NotFound':
             return f'{emd}: EMD_Error'
         if len(jibun) < 1:
             jibun = '0'
         # 대장구분
         mt = '2' if jibun.startswith('산') else '1'
         # 지번에서 공백 및 '산' 제거
         jibun = jibun.replace(' ','').strip('산')
         # 지번지목으로 입력시 지목 제거
         jibun = jibun if jibun[-1].isnumeric() else jibun[:-1]
         # 부번이 없으면 '-0' 추가
         jibun = jibun if '-' in jibun else jibun + '-0'
         # 본번, 부번 분리
         bon, bu = jibun.split('-')

         # PNU 생성 반환
         return ''.join([cd, mt, bon.zfill(4), bu.zfill(4)])
         
      
if __name__ == '__main__':
    d = DistrictCode()
    ser = d.search('팔달구 인계')
    pnu = d.make_pnu('팔달구 인', '')
    emd = d.emd('4111514100209810001')
    print(ser, pnu, emd)